In [1]:
import pandas as pd
import json
import os
from tqdm import tqdm

# Parlamentares

In [9]:
files = os.listdir('./data/parlamentares_detalhes/')

data = []
for file in tqdm(files):
    with open(f"./data/parlamentares_detalhes/{file}", 'r') as f:
        parl = json.load(f)
    try:
        d = [{**d, **d['ultimoStatus']} for d in parl]
    except KeyError:
        continue
    try:
        d[0].pop('ultimoStatus')
    except IndexError:
        continue
    data += d

df_parls = pd.DataFrame(data)

100%|██████████| 3093/3093 [00:00<00:00, 6622.94it/s]


In [10]:
df_parls.to_csv('./data/dim_parlamentares.csv')

# Votos

In [11]:
print('Carregando votações...')
votacoes_files = os.listdir('./data/votacoes')
votacoes_files = [
    file for file in votacoes_files if '2019' in file or '2020' in file or '2021' in file or '2022' in file]

votacoes = []
for votacao in tqdm(votacoes_files):
    with open('./data/votacoes/' + votacao, 'r') as f:
        data = json.load(f)
    votacoes += data
print(f'Total de votações: {len(votacoes)}')

print('Carregando votos...')
votacoes_ids = [vot['id'] for vot in votacoes]

votos_files = os.listdir('./data/votos/')
votos_files = [file for file in votos_files if file.split('.')[
    0] in votacoes_ids]

votos = []
for v_vile in tqdm(votos_files):
    with open('./data/votos/' + v_vile, 'r') as f:
        data = json.load(f)
    if len(data) == 0:
        continue
    id_votacao = v_vile.split('.')[0]
    data = d = [{'id_votacao': id_votacao, 'tipoVoto': d['tipoVoto'],
                 'dataRegistroVoto': d['dataRegistroVoto'], **d['deputado_']} for d in data]
    votos += data

print(f'Total de votos: {len(votos)}')

df_votos = pd.DataFrame(votos)

Carregando votações...


100%|██████████| 1461/1461 [00:00<00:00, 12720.22it/s]

Total de votações: 25789
Carregando votos...



100%|██████████| 25789/25789 [00:06<00:00, 3752.28it/s]


Total de votos: 800095


In [12]:
df_votos.to_csv('./data/fat_votos_2019-2022.csv', index=False)

# Profissões

In [13]:
print('Carregando profissões...')
ls_ids_parls_votos = df_votos['id'].unique()
ls_ids_parls_votos = [str(p) for p in ls_ids_parls_votos]

parls_profs = os.listdir('./data/parlamentares_profissoes/')
parls_profs = [p for p in parls_profs if p.split('.')[0] in ls_ids_parls_votos]

profs = []
parls_sem_prof = 0
for file in tqdm(parls_profs):
    with open(f"./data/parlamentares_profissoes/{file}", 'r') as f:
        data = json.load(f)
    if len(data) == 0:
        parls_sem_prof == 1
        continue
    id_parl = file.split('.')[0]
    data = [{'id_parl': id_parl, **d} for d in data]
    profs += data

print(f'Total de profissões: {len(profs)} \nParlamentares sem profissões registradas: {parls_sem_prof}')
df_profs = pd.DataFrame(profs)

df_dim_profs = pd.read_excel('./data/dim_profissoes_ficha.xlsx')

df_profs_m = df_profs.merge(df_dim_profs[['codTipoProfissao', 'is_empresario']], how='left', on='codTipoProfissao')

Carregando profissões...


100%|██████████| 597/597 [00:00<00:00, 7070.57it/s]

Total de profissões: 802 
Parlamentares sem profissões registradas: 0


In [21]:
df_profs_m.to_csv('./data/fat_profissoes_2019-2023.csv', index=False)

# Orientações

In [15]:
print('Carregando orientações...')
orient_files = os.listdir('./data/orientacoes/')
orient_files = [file for file in orient_files if file.split('.')[
    0] in votacoes_ids]

orientacoes = []
for ori_vile in tqdm(orient_files):
    with open('./data/orientacoes/' + ori_vile, 'r') as f:
        data = json.load(f)
    if len(data) == 0:
        continue
    id_votacao = ori_vile.split('.')[0]
    data = d = [{'id_votacao': id_votacao, **d} for d in data]
    orientacoes += data

print(f'Total de orientacoes: {len(orientacoes)}')

df_orientacoes = pd.DataFrame(orientacoes)

govs = [b for b in df_orientacoes['siglaPartidoBloco'].unique() if 'Gov' in b]
df_orient_gov = df_orientacoes[df_orientacoes['siglaPartidoBloco'].isin(govs)].reset_index(drop=True)

Carregando orientações...


100%|██████████| 25789/25789 [00:02<00:00, 8786.28it/s]

Total de orientacoes: 53213


In [16]:
df_orientacoes.to_csv('./data/fat_orientacoes_2019-2022.csv', index=False)
df_orient_gov.to_csv('./data/fat_orientacoes_gov_2019-2022.csv', index=False)

## Verificar sinal

In [17]:
from enum import Enum
# from core.tipo_voto import TipoVoto

In [18]:
class TipoVoto(Enum):
    SIM = 1
    ABSTENÇÃO = 0
    NÃO = -1
    OBSTRUÇÃO = -1 
    ARTIGO_17 = 0
    NONE = 0
    FAVORÁVEL_COM_RESTRIÇÕES = 1 
    BRANCO = 0
    LIBERADO = 0

    @classmethod
    def trat_voto(cls, voto: str) -> int:
        voto_trat = str(voto).upper().replace(' ', '_')
        if voto_trat == 'NAN' or voto_trat == '':
            return None
        return cls[voto_trat].value

In [19]:
df_v_o = df_votos.merge(df_orient_gov, on='id_votacao', how='left')
df_v_o['voto_trat'] = df_v_o['tipoVoto'].apply(lambda x: TipoVoto.trat_voto(x))
df_v_o['orient_trat'] = df_v_o['orientacaoVoto'].apply(lambda x: TipoVoto.trat_voto(x))

df_v_o['gov'] = df_v_o['voto_trat'] * df_v_o['orient_trat']

df_v_o['ano'] = [date.split('-')[0] for date in df_v_o['dataRegistroVoto']]

In [20]:
df_v_o.to_csv('./data/fat_votos_ori_gov_2019-2022.csv', index=False)